# Advanced Querying and Performance

## Learning Objectives
- Master advanced SQLAlchemy querying techniques
- Learn query optimization and performance tuning
- Understand eager loading strategies and N+1 problem solutions
- Master complex joins, subqueries, and window functions
- Learn query profiling and performance monitoring
- Practice real-world query optimization scenarios

## What You'll Learn
- Advanced querying with joins, subqueries, and unions
- Query optimization techniques and best practices
- Eager loading strategies (joinedload, subqueryload, selectinload)
- Query profiling and performance analysis
- Database indexing and query hints
- Caching strategies and connection pooling


## Advanced Querying Overview

Advanced querying in SQLAlchemy involves:

- **Complex Joins** - Multiple table relationships and join types
- **Subqueries** - Nested queries for complex data retrieval
- **Window Functions** - Advanced analytical functions
- **Query Optimization** - Performance tuning and efficiency
- **Eager Loading** - Preventing N+1 query problems
- **Query Profiling** - Analyzing and monitoring performance

## Performance Considerations

- **N+1 Problem** - Loading related objects inefficiently
- **Query Optimization** - Reducing database round trips
- **Indexing** - Proper database indexing strategies
- **Connection Pooling** - Managing database connections
- **Caching** - Reducing database load with caching


## 1. Setup and Model Definition

Let's start by setting up our environment and defining models for advanced querying examples:


In [ ]:
# Setup
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float, Index
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, joinedload, subqueryload, selectinload
from sqlalchemy import func, and_, or_, not_, desc, asc
from datetime import datetime, timedelta
import time

# Create database engine with echo for SQL logging
engine = create_engine('sqlite:///advanced_querying.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

print("SQLAlchemy setup complete!")


In [ ]:
# Define models for advanced querying examples
class Author(Base):
    __tablename__ = 'authors'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, index=True)
    email = Column(String(100), unique=True, nullable=False)
    bio = Column(Text)
    birth_date = Column(DateTime)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    books = relationship("Book", back_populates="author", cascade="all, delete-orphan")
    reviews = relationship("Review", back_populates="reviewer")
    
    def __repr__(self):
        return f"<Author(name='{self.name}', email='{self.email}')>"

class Category(Base):
    __tablename__ = 'categories'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False, unique=True)
    description = Column(Text)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    books = relationship("Book", back_populates="category")
    
    def __repr__(self):
        return f"<Category(name='{self.name}')>"

class Book(Base):
    __tablename__ = 'books'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False, index=True)
    isbn = Column(String(20), unique=True, nullable=False)
    published_year = Column(Integer, index=True)
    price = Column(Float, nullable=False)
    page_count = Column(Integer)
    is_available = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    author_id = Column(Integer, ForeignKey('authors.id'), nullable=False)
    category_id = Column(Integer, ForeignKey('categories.id'), nullable=False)
    
    # Relationships
    author = relationship("Author", back_populates="books")
    category = relationship("Category", back_populates="books")
    reviews = relationship("Review", back_populates="book", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Book(title='{self.title}', author_id={self.author_id})>"

class Review(Base):
    __tablename__ = 'reviews'
    
    id = Column(Integer, primary_key=True)
    rating = Column(Integer, nullable=False)  # 1-5 stars
    comment = Column(Text)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    book_id = Column(Integer, ForeignKey('books.id'), nullable=False)
    reviewer_id = Column(Integer, ForeignKey('authors.id'), nullable=False)
    
    # Relationships
    book = relationship("Book", back_populates="reviews")
    reviewer = relationship("Author", back_populates="reviews")
    
    def __repr__(self):
        return f"<Review(rating={self.rating}, book_id={self.book_id})>"

# Create indexes for performance
Index('idx_books_author_category', Book.author_id, Book.category_id)
Index('idx_reviews_book_rating', Review.book_id, Review.rating)

# Create tables
Base.metadata.create_all(engine)

print("Models defined and tables created!")
print("Models: Author, Category, Book, Review")
print("Indexes created for performance optimization")


In [ ]:
# Create sample data for advanced querying examples
session = Session()

# Create categories
categories_data = [
    Category(name="Fiction", description="Novels and short stories"),
    Category(name="Non-Fiction", description="Biographies, history, science"),
    Category(name="Technology", description="Programming and technical books"),
    Category(name="Mystery", description="Crime and detective stories"),
    Category(name="Romance", description="Love stories and relationships")
]

session.add_all(categories_data)
session.commit()

# Create authors
authors_data = [
    Author(name="J.K. Rowling", email="jkrowling@example.com", bio="Author of Harry Potter series", birth_date=datetime(1965, 7, 31)),
    Author(name="George Orwell", email="gorwell@example.com", bio="Author of 1984 and Animal Farm", birth_date=datetime(1903, 6, 25)),
    Author(name="Jane Austen", email="jausten@example.com", bio="Author of Pride and Prejudice", birth_date=datetime(1775, 12, 16)),
    Author(name="Mark Twain", email="mtwain@example.com", bio="Author of Adventures of Huckleberry Finn", birth_date=datetime(1835, 11, 30)),
    Author(name="Agatha Christie", email="achristie@example.com", bio="Mystery writer", birth_date=datetime(1890, 9, 15)),
    Author(name="Robert Martin", email="rmartin@example.com", bio="Software engineering expert", birth_date=datetime(1952, 3, 5))
]

session.add_all(authors_data)
session.commit()

# Create books
books_data = [
    Book(title="Harry Potter and the Philosopher's Stone", isbn="978-0747532699", published_year=1997, price=12.99, page_count=223, author_id=1, category_id=1),
    Book(title="Harry Potter and the Chamber of Secrets", isbn="978-0747538493", published_year=1998, price=13.99, page_count=251, author_id=1, category_id=1),
    Book(title="1984", isbn="978-0451524935", published_year=1949, price=10.99, page_count=328, author_id=2, category_id=2),
    Book(title="Animal Farm", isbn="978-0451526342", published_year=1945, price=9.99, page_count=112, author_id=2, category_id=2),
    Book(title="Pride and Prejudice", isbn="978-0141439518", published_year=1813, price=11.99, page_count=432, author_id=3, category_id=5),
    Book(title="Adventures of Huckleberry Finn", isbn="978-0486280615", published_year=1884, price=14.99, page_count=366, author_id=4, category_id=1),
    Book(title="Murder on the Orient Express", isbn="978-0062073495", published_year=1934, price=15.99, page_count=288, author_id=5, category_id=4),
    Book(title="Clean Code", isbn="978-0132350884", published_year=2008, price=45.99, page_count=464, author_id=6, category_id=3),
    Book(title="The Pragmatic Programmer", isbn="978-0201616224", published_year=1999, price=39.99, page_count=352, author_id=6, category_id=3)
]

session.add_all(books_data)
session.commit()

# Create reviews
reviews_data = [
    Review(rating=5, comment="Amazing book!", book_id=1, reviewer_id=2),
    Review(rating=4, comment="Great story", book_id=1, reviewer_id=3),
    Review(rating=5, comment="Classic dystopian novel", book_id=3, reviewer_id=1),
    Review(rating=4, comment="Thought-provoking", book_id=3, reviewer_id=4),
    Review(rating=5, comment="Timeless romance", book_id=5, reviewer_id=2),
    Review(rating=3, comment="Good but slow", book_id=5, reviewer_id=5),
    Review(rating=4, comment="Excellent mystery", book_id=7, reviewer_id=1),
    Review(rating=5, comment="Must-read for developers", book_id=8, reviewer_id=2),
    Review(rating=5, comment="Practical advice", book_id=8, reviewer_id=3),
    Review(rating=4, comment="Good programming practices", book_id=9, reviewer_id=1)
]

session.add_all(reviews_data)
session.commit()

print("Sample data created successfully!")
print(f"Categories: {len(categories_data)}")
print(f"Authors: {len(authors_data)}")
print(f"Books: {len(books_data)}")
print(f"Reviews: {len(reviews_data)}")


## 2. Advanced Joins and Complex Queries

Let's explore advanced join techniques and complex query patterns:


In [ ]:
# Advanced joins with multiple tables
print("=== Advanced Joins ===")

# 1. Multiple table joins
print("\n1. Books with Author and Category information:")
books_with_details = session.query(Book, Author, Category).join(Author).join(Category).all()

for book, author, category in books_with_details:
    print(f"  {book.title} by {author.name} ({category.name}) - ${book.price}")

# 2. Left outer join to include books without reviews
print("\n2. All books with review count (including books with no reviews):")
from sqlalchemy.orm import outerjoin

books_with_review_count = session.query(
    Book.title,
    Author.name,
    func.count(Review.id).label('review_count')
).join(Author).outerjoin(Review).group_by(Book.id, Book.title, Author.name).all()

for title, author_name, review_count in books_with_review_count:
    print(f"  {title} by {author_name} - {review_count} reviews")

# 3. Complex join with conditions
print("\n3. Books with high ratings (4+ stars) and their details:")
high_rated_books = session.query(
    Book.title,
    Author.name,
    Category.name.label('category'),
    func.avg(Review.rating).label('avg_rating'),
    func.count(Review.id).label('review_count')
).join(Author).join(Category).join(Review).group_by(
    Book.id, Book.title, Author.name, Category.name
).having(func.avg(Review.rating) >= 4.0).all()

for title, author, category, avg_rating, review_count in high_rated_books:
    print(f"  {title} by {author} ({category}) - Avg: {avg_rating:.1f} ({review_count} reviews)")

# 4. Self-join example (authors who reviewed their own books)
print("\n4. Authors who reviewed their own books:")
self_reviews = session.query(
    Author.name,
    Book.title,
    Review.rating
).join(Book, Author.id == Book.author_id).join(
    Review, and_(Review.book_id == Book.id, Review.reviewer_id == Author.id)
).all()

for author_name, book_title, rating in self_reviews:
    print(f"  {author_name} reviewed '{book_title}' with {rating} stars")


## 3. Subqueries and Advanced Filtering

Let's explore subqueries and complex filtering techniques:


In [ ]:
# Subqueries and advanced filtering
print("=== Subqueries and Advanced Filtering ===")

# 1. Subquery to find books with above-average ratings
print("\n1. Books with above-average ratings:")
avg_rating_subquery = session.query(func.avg(Review.rating)).scalar()

above_avg_books = session.query(Book).join(Review).group_by(Book.id).having(
    func.avg(Review.rating) > avg_rating_subquery
).all()

print(f"Average rating across all books: {avg_rating_subquery:.2f}")
for book in above_avg_books:
    print(f"  {book.title}")

# 2. EXISTS subquery to find authors who have written books
print("\n2. Authors who have written books (using EXISTS):")
authors_with_books = session.query(Author).filter(
    session.query(Book).filter(Book.author_id == Author.id).exists()
).all()

for author in authors_with_books:
    print(f"  {author.name}")

# 3. IN subquery to find books in specific categories
print("\n3. Books in Fiction or Technology categories:")
fiction_tech_categories = session.query(Category.id).filter(
    Category.name.in_(['Fiction', 'Technology'])
).subquery()

books_in_categories = session.query(Book).filter(
    Book.category_id.in_(session.query(fiction_tech_categories.c.id))
).all()

for book in books_in_categories:
    print(f"  {book.title} ({book.category.name})")

# 4. Correlated subquery to find authors with more than 1 book
print("\n4. Authors with more than 1 book:")
authors_multiple_books = session.query(Author).filter(
    session.query(func.count(Book.id)).filter(Book.author_id == Author.id).scalar() > 1
).all()

for author in authors_multiple_books:
    book_count = session.query(func.count(Book.id)).filter(Book.author_id == author.id).scalar()
    print(f"  {author.name} ({book_count} books)")

# 5. Complex filtering with multiple conditions
print("\n5. Books published after 1900, priced under $20, with 4+ star average rating:")
complex_filter_books = session.query(Book).join(Review).filter(
    and_(
        Book.published_year > 1900,
        Book.price < 20.0
    )
).group_by(Book.id).having(func.avg(Review.rating) >= 4.0).all()

for book in complex_filter_books:
    avg_rating = session.query(func.avg(Review.rating)).filter(Review.book_id == book.id).scalar()
    print(f"  {book.title} (${book.price}, {book.published_year}, {avg_rating:.1f}★)")


## 4. Eager Loading and N+1 Problem

The N+1 problem occurs when you load a collection of objects and then access a related object for each one, resulting in N+1 database queries. Let's explore eager loading strategies:


In [ ]:
# Demonstrate N+1 problem and eager loading solutions
print("=== N+1 Problem and Eager Loading ===")

# 1. N+1 Problem Example (BAD)
print("\n1. N+1 Problem Example (BAD - Multiple Queries):")
print("Loading books and then accessing author for each book...")

# Turn off echo to see the difference
engine.echo = False

start_time = time.time()
books = session.query(Book).all()  # 1 query
for book in books:
    print(f"  {book.title} by {book.author.name}")  # N queries (one per book)
n_plus_1_time = time.time() - start_time

print(f"Time taken: {n_plus_1_time:.4f} seconds")
print(f"Queries executed: {len(books) + 1} (1 + {len(books)})")

# 2. Solution 1: joinedload (Single Query with JOIN)
print("\n2. Solution 1: joinedload (Single Query with JOIN):")
start_time = time.time()
books_with_authors = session.query(Book).options(joinedload(Book.author)).all()  # 1 query
for book in books_with_authors:
    print(f"  {book.title} by {book.author.name}")
joinedload_time = time.time() - start_time

print(f"Time taken: {joinedload_time:.4f} seconds")
print("Queries executed: 1 (with JOIN)")

# 3. Solution 2: subqueryload (Two Queries)
print("\n3. Solution 2: subqueryload (Two Queries):")
start_time = time.time()
books_with_authors_subquery = session.query(Book).options(subqueryload(Book.author)).all()  # 2 queries
for book in books_with_authors_subquery:
    print(f"  {book.title} by {book.author.name}")
subqueryload_time = time.time() - start_time

print(f"Time taken: {subqueryload_time:.4f} seconds")
print("Queries executed: 2 (main query + subquery)")

# 4. Solution 3: selectinload (Two Queries, Better for Large Collections)
print("\n4. Solution 3: selectinload (Two Queries, Better for Large Collections):")
start_time = time.time()
books_with_authors_selectin = session.query(Book).options(selectinload(Book.author)).all()  # 2 queries
for book in books_with_authors_selectin:
    print(f"  {book.title} by {book.author.name}")
selectinload_time = time.time() - start_time

print(f"Time taken: {selectinload_time:.4f} seconds")
print("Queries executed: 2 (main query + SELECT IN)")

# 5. Multiple eager loading
print("\n5. Multiple Eager Loading (Author + Category + Reviews):")
books_with_all = session.query(Book).options(
    joinedload(Book.author),
    joinedload(Book.category),
    joinedload(Book.reviews)
).all()

for book in books_with_all:
    print(f"  {book.title} by {book.author.name} ({book.category.name}) - {len(book.reviews)} reviews")

# Turn echo back on
engine.echo = True

print(f"\nPerformance Comparison:")
print(f"  N+1 Problem: {n_plus_1_time:.4f}s")
print(f"  joinedload:   {joinedload_time:.4f}s")
print(f"  subqueryload: {subqueryload_time:.4f}s")
print(f"  selectinload: {selectinload_time:.4f}s")


## 5. Window Functions and Advanced Analytics

Window functions allow you to perform calculations across a set of rows related to the current row:


In [ ]:
# Window functions and advanced analytics
print("=== Window Functions and Advanced Analytics ===")

from sqlalchemy import over

# 1. ROW_NUMBER() - Rank books by price within each category
print("\n1. Books ranked by price within each category:")
price_rank_query = session.query(
    Book.title,
    Category.name.label('category'),
    Book.price,
    func.row_number().over(
        partition_by=Category.name,
        order_by=Book.price.desc()
    ).label('price_rank')
).join(Category).all()

for title, category, price, rank in price_rank_query:
    print(f"  #{rank} {title} ({category}) - ${price}")

# 2. RANK() - Rank authors by number of books
print("\n2. Authors ranked by number of books:")
author_rank_query = session.query(
    Author.name,
    func.count(Book.id).label('book_count'),
    func.rank().over(order_by=func.count(Book.id).desc()).label('author_rank')
).join(Book).group_by(Author.id, Author.name).all()

for name, book_count, rank in author_rank_query:
    print(f"  #{rank} {name} - {book_count} books")

# 3. LAG() - Compare book prices with previous book in same category
print("\n3. Book prices compared to previous book in same category:")
price_comparison_query = session.query(
    Book.title,
    Category.name.label('category'),
    Book.price,
    func.lag(Book.price).over(
        partition_by=Category.name,
        order_by=Book.price
    ).label('previous_price')
).join(Category).all()

for title, category, price, prev_price in price_comparison_query:
    if prev_price:
        diff = price - prev_price
        print(f"  {title} ({category}) - ${price} (${diff:+.2f} vs previous)")
    else:
        print(f"  {title} ({category}) - ${price} (first in category)")

# 4. Running totals and cumulative functions
print("\n4. Running total of book prices by publication year:")
running_total_query = session.query(
    Book.published_year,
    Book.title,
    Book.price,
    func.sum(Book.price).over(
        order_by=Book.published_year,
        rows=(None, 0)  # Unbounded preceding to current row
    ).label('running_total')
).order_by(Book.published_year).all()

for year, title, price, total in running_total_query:
    print(f"  {year}: {title} - ${price} (Total: ${total:.2f})")

# 5. Percentile calculations
print("\n5. Books with percentile rankings by price:")
percentile_query = session.query(
    Book.title,
    Book.price,
    func.percent_rank().over(order_by=Book.price).label('price_percentile'),
    func.ntile(4).over(order_by=Book.price).label('price_quartile')
).all()

for title, price, percentile, quartile in percentile_query:
    print(f"  {title} - ${price} (Top {100-percentile*100:.0f}%, Quartile {quartile})")


## 6. Query Optimization and Performance

Let's explore query optimization techniques and performance monitoring:


In [ ]:
# Query optimization and performance techniques
print("=== Query Optimization and Performance ===")

# 1. Query profiling with timing
print("\n1. Query Performance Comparison:")

# Turn off echo for cleaner output
engine.echo = False

# Slow query (without optimization)
start_time = time.time()
slow_books = session.query(Book).all()
for book in slow_books:
    author_name = book.author.name
    category_name = book.category.name
    review_count = len(book.reviews)
slow_time = time.time() - start_time

# Optimized query (with eager loading)
start_time = time.time()
optimized_books = session.query(Book).options(
    joinedload(Book.author),
    joinedload(Book.category),
    joinedload(Book.reviews)
).all()
for book in optimized_books:
    author_name = book.author.name
    category_name = book.category.name
    review_count = len(book.reviews)
optimized_time = time.time() - start_time

print(f"  Slow query (N+1): {slow_time:.4f} seconds")
print(f"  Optimized query:  {optimized_time:.4f} seconds")
print(f"  Performance gain: {slow_time/optimized_time:.1f}x faster")

# 2. Using indexes effectively
print("\n2. Index Usage Examples:")
print("  - Books by title (indexed):")
start_time = time.time()
books_by_title = session.query(Book).filter(Book.title.like('%Harry%')).all()
index_time = time.time() - start_time
print(f"    Found {len(books_by_title)} books in {index_time:.4f}s")

print("  - Books by published year (indexed):")
start_time = time.time()
books_by_year = session.query(Book).filter(Book.published_year > 1990).all()
year_time = time.time() - start_time
print(f"    Found {len(books_by_year)} books in {year_time:.4f}s")

# 3. Query result caching simulation
print("\n3. Query Result Caching (Simulation):")
cache = {}

def cached_query(query_key, query_func):
    if query_key in cache:
        print(f"  Cache hit for '{query_key}'")
        return cache[query_key]
    else:
        print(f"  Cache miss for '{query_key}' - executing query")
        result = query_func()
        cache[query_key] = result
        return result

# First call - cache miss
start_time = time.time()
expensive_books = cached_query("expensive_books", 
    lambda: session.query(Book).filter(Book.price > 30).all())
first_call_time = time.time() - start_time

# Second call - cache hit
start_time = time.time()
expensive_books_cached = cached_query("expensive_books",
    lambda: session.query(Book).filter(Book.price > 30).all())
second_call_time = time.time() - start_time

print(f"  First call: {first_call_time:.4f}s")
print(f"  Cached call: {second_call_time:.4f}s")

# 4. Pagination for large result sets
print("\n4. Pagination Example:")
def paginate_books(page=1, per_page=3):
    offset = (page - 1) * per_page
    books = session.query(Book).offset(offset).limit(per_page).all()
    total_count = session.query(Book).count()
    total_pages = (total_count + per_page - 1) // per_page
    
    return books, total_count, total_pages

for page in range(1, 4):
    books, total, pages = paginate_books(page, 3)
    print(f"  Page {page}/{pages} ({len(books)} books):")
    for book in books:
        print(f"    {book.title}")

# Turn echo back on
engine.echo = True


## 7. Best Practices and Common Pitfalls

Let's cover best practices and common mistakes to avoid:


In [ ]:
# Best practices and common pitfalls
print("=== Best Practices and Common Pitfalls ===")

print("\n1. ✅ DO: Use eager loading to prevent N+1 queries")
print("   ❌ DON'T: Access relationships in loops without eager loading")
print("   Example:")
print("   # BAD: N+1 queries")
print("   books = session.query(Book).all()")
print("   for book in books:")
print("       print(book.author.name)  # N queries!")
print("   ")
print("   # GOOD: Single query with eager loading")
print("   books = session.query(Book).options(joinedload(Book.author)).all()")
print("   for book in books:")
print("       print(book.author.name)  # No additional queries!")

print("\n2. ✅ DO: Use appropriate indexes")
print("   ❌ DON'T: Query unindexed columns frequently")
print("   Example:")
print("   # Add indexes for frequently queried columns")
print("   class Book(Base):")
print("       title = Column(String(200), index=True)  # ✅ Indexed")
print("       published_year = Column(Integer, index=True)  # ✅ Indexed")

print("\n3. ✅ DO: Use pagination for large result sets")
print("   ❌ DON'T: Load all records at once")
print("   Example:")
print("   # BAD: Load all books")
print("   all_books = session.query(Book).all()  # Could be millions!")
print("   ")
print("   # GOOD: Use pagination")
print("   books = session.query(Book).offset(0).limit(20).all()")

print("\n4. ✅ DO: Use query optimization techniques")
print("   ❌ DON'T: Write inefficient queries")
print("   Example:")
print("   # BAD: Multiple queries")
print("   for author in authors:")
print("       books = session.query(Book).filter(Book.author_id == author.id).all()")
print("   ")
print("   # GOOD: Single query with join")
print("   books_with_authors = session.query(Book, Author).join(Author).all()")

print("\n5. ✅ DO: Use connection pooling")
print("   ❌ DON'T: Create new connections for each query")
print("   Example:")
print("   # GOOD: Reuse session")
print("   session = Session()")
print("   # ... multiple queries ...")
print("   session.close()")

print("\n6. ✅ DO: Use transactions appropriately")
print("   ❌ DON'T: Commit after every single operation")
print("   Example:")
print("   # BAD: Multiple commits")
print("   session.add(book1); session.commit()")
print("   session.add(book2); session.commit()")
print("   ")
print("   # GOOD: Batch operations")
print("   session.add_all([book1, book2, book3])")
print("   session.commit()")

print("\n7. ✅ DO: Use query result caching for expensive operations")
print("   ❌ DON'T: Re-execute expensive queries unnecessarily")
print("   Example:")
print("   # Cache expensive aggregation queries")
print("   cache_key = 'book_stats'")
print("   if cache_key not in cache:")
print("       stats = session.query(func.avg(Book.price)).scalar()")
print("       cache[cache_key] = stats")

print("\n8. ✅ DO: Profile and monitor query performance")
print("   ❌ DON'T: Ignore slow queries")
print("   Example:")
print("   import time")
print("   start = time.time()")
print("   result = session.query(Book).all()")
print("   print(f'Query took {time.time() - start:.4f} seconds')")

# Close the session
session.close()

print("\n" + "="*50)
print("Advanced Querying and Performance examples completed!")
print("="*50)


## Summary

You've learned about:

1. **Advanced Joins** - Multiple table joins, outer joins, and complex join conditions
2. **Subqueries** - EXISTS, IN, and correlated subqueries for complex filtering
3. **Eager Loading** - joinedload, subqueryload, and selectinload to prevent N+1 problems
4. **Window Functions** - ROW_NUMBER, RANK, LAG, and analytical functions
5. **Query Optimization** - Performance profiling, indexing, and caching strategies
6. **Best Practices** - Common pitfalls and optimization techniques

## Key Takeaways

- **N+1 Problem**: Use eager loading to prevent multiple database queries
- **Query Optimization**: Profile queries and use appropriate indexes
- **Window Functions**: Powerful for analytical queries and rankings
- **Performance**: Monitor query performance and use caching when appropriate
- **Best Practices**: Follow established patterns for efficient database operations

## Performance Tips

- Always use eager loading for related objects
- Add indexes for frequently queried columns
- Use pagination for large result sets
- Profile and monitor query performance
- Cache expensive query results
- Use connection pooling effectively
- Batch database operations when possible

Ready to practice? Move on to the exercise notebook!
